In [ ]:
import uproot
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats
import pylandau
from scipy.interpolate import CubicSpline

In [ ]:
savedir = "/icarus/app/users/gputnam/calib/plots/"
dosave=False

In [ ]:
branches = [
    "meta.run",
    "meta.evt",
    "meta.iproc",
    "t0",
    "id",
    "clear_cosmic_muon",
    "start_x",
    "start_y",
    "start_z",
    "end_x",
    "end_y",
    "end_z",
    "dir_x",
    "dir_y",
    "dir_z",
    "hit_min_time_p2_tpcE",
    "hit_max_time_p2_tpcE",
    "hit_min_time_p2_tpcW",
    "hit_max_time_p2_tpcW"
]

In [ ]:
# fname = "icarus.root"
fname = "/icarus/app/users/gputnam/calib/proc/caloanaout_run5458_N1000_procF_stop.root"
folderE = "caloanaCryoE"
folderW = "caloanaCryoW"
tname = "GCalorimetryAnalyzer"

treeE = uproot.open(fname)[folderE][tname]
data = treeE.arrays(branches, library="pd")

In [ ]:
# EXTERNAL INPUT: The drift window in TPC E and TPC W
tcathode_E = 3203.15
tcathode_W = 3212.96

# EXTERNAL INPUT: electron lifetime in TPC E and TPC W
electron_lifetime_tpcE = 3.38e3
electron_lifetime_tpcW = 3.65e3

# EXTERNAL INPUT: calibration constant
calibration = 81.01
MEAN_PITCH = 0.3487

In [ ]:
# CONFIG

a2c_dist = 148.275 # cm
tanode = 850 # ticks
tick_period = 400 # ns
NTICKS = 4096 # ticks

In [ ]:
# Compute predicted R.R. v. dE/dx
mass_electron = 0.5109989461 # MeV https://pdg.lbl.gov/2020/listings/rpp2020-list-K-plus-minus.pdf
mass = 105.6583745 # MeV https://pdg.lbl.gov/2020/listings/rpp2020-list-muon.pdf
Ival = 188.0e-6
Zval = 18.0
Aval = 39.948
Kfactor = 0.307075
LAr_density_gmL = 1.3973

def Calc_MPV_DEDX(pitch, T):
    gamma = (mass+T)/mass
    beta = pow(1.0-pow(gamma,-2.0),0.5)
    Wmax = (2.0*mass_electron*pow(beta,2.0)*pow(gamma,2.0))/(1.0+2.0*gamma*(mass_electron/mass)+pow(mass_electron/mass,2.0))

    # Medium energy 
    dens_factor = 2.0*np.log(10)*np.log10(beta*gamma)-5.2146+0.19559*pow(3.0-np.log10(beta*gamma),3.0)
    # low energy
    dens_factor[np.log10(beta*gamma) < 0.2] = 0.
    # high energy
    dens_factor[np.log10(beta*gamma) > 3.0] = (2.0*np.log(10)*np.log10(beta*gamma)-5.2146)[np.log10(beta*gamma) > 3.0]
    dEdx_mean = LAr_density_gmL*Kfactor*(Zval/Aval)*pow(beta,-2.0)*(0.5*np.log(2.0*mass_electron*pow(beta,2.0)*pow(gamma,2.0)*Wmax*pow(Ival,-2.0))-pow(beta,2.0)-dens_factor/2.0)
    xi = (Kfactor/2.0)*(Zval/Aval)*pow(beta,-2.0)*LAr_density_gmL*pitch
    kappa = xi/Wmax
    dEdx_MPV = xi*(np.log((2.0*mass_electron*pow(beta*gamma,2.0))/Ival)+np.log(xi/Ival)+0.200-pow(beta,2.0)-dens_factor)/pitch
  
    return dEdx_MPV

def Calc_MEAN_DEDX(T):
    gamma = (mass+T)/mass
    beta = pow(1.0-pow(gamma,-2.0),0.5)
    Wmax = (2.0*mass_electron*pow(beta,2.0)*pow(gamma,2.0))/(1.0+2.0*gamma*(mass_electron/mass)+pow(mass_electron/mass,2.0))

    # Medium energy 
    dens_factor = 2.0*np.log(10)*np.log10(beta*gamma)-5.2146+0.19559*pow(3.0-np.log10(beta*gamma),3.0)
    # low energy
    dens_factor[np.log10(beta*gamma) < 0.2] = 0.
    dens_factor[beta < 1e-6] = 0.
    # high energy
    dens_factor[np.log10(beta*gamma) > 3.0] = (2.0*np.log(10)*np.log10(beta*gamma)-5.2146)[np.log10(beta*gamma) > 3.0]
    dEdx_mean = LAr_density_gmL*Kfactor*(Zval/Aval)*pow(beta,-2.0)*(0.5*np.log(2.0*mass_electron*pow(beta,2.0)*pow(gamma,2.0)*Wmax*pow(Ival,-2.0))-pow(beta,2.0)-dens_factor/2.0)

    return dEdx_mean

In [ ]:
KE_points_max = 1000.
dRR = 0.01
thisKE = KE_points_max

KE_points = [thisKE]
RR_points = [0.]

while thisKE > 0.0:
    deltaKE = Calc_MEAN_DEDX(np.array([thisKE])) * dRR
    RR_points.append(RR_points[-1] + dRR)
    thisKE -= deltaKE[0]
    KE_points.append(thisKE)
    
KE_points = np.array(list(reversed(KE_points[:-1])))
RR_points = np.array(RR_points[:-1])

In [ ]:
MPV_dEDX_points = Calc_MPV_DEDX(MEAN_PITCH, KE_points)

RR2DEDX = CubicSpline(RR_points, MPV_dEDX_points)

In [ ]:
ICARUS_AV = [    
    {
      "xmin": -364.49,
      "xmax": -67.94,
      "ymin": -173.41,
      "zmin": -909.951,
      "ymax": 143.41,
      "zmax": 879.951,
    },
    {
      "xmin": 67.94,
      "ymin": -173.41,
      "zmin": -909.951,
      "xmax": 364.49,
      "ymax": 143.41,
      "zmax": 879.951,
    }
]

ICARUS_FV = [{
        "xmin": ICARUS_AV[0]["xmin"] + 25,
        "ymin": ICARUS_AV[0]["ymin"] + 25,
        "zmin": ICARUS_AV[0]["zmin"] + 25,
        "xmax": ICARUS_AV[0]["xmax"] - 25,
        "ymax": ICARUS_AV[0]["ymax"] - 50,
        "zmax": ICARUS_AV[0]["zmax"] - 25,
    },
    {
        "xmin": ICARUS_AV[1]["xmin"] + 25,
        "ymin": ICARUS_AV[1]["ymin"] + 25,
        "zmin": ICARUS_AV[1]["zmin"] + 25,
        "xmax": ICARUS_AV[1]["xmax"] - 25,
        "ymax": ICARUS_AV[1]["ymax"] - 50,
        "zmax": ICARUS_AV[1]["zmax"] - 25,
    }
]

def InFV(xyz, FVs=ICARUS_FV):
    fids = []
    fid = xyz[0] == np.nan
    for fv in FVs:
        this_fid = [(xyz[i] > fv[p+"min"]) & (xyz[i] < fv[p+"max"]) for i, p in enumerate("xyz")]
        fid = fid | (this_fid[0] & this_fid[1] & this_fid[2])
    return fid

def InFVYZ(yz, FVs=ICARUS_FV):
    fids = []
    fid = yz[0] == np.nan
    for fv in FVs:
        this_fid = [(yz[i] > fv[p+"min"]) & (yz[i] < fv[p+"max"]) for i, p in enumerate("yz")]
        fid = fid | (this_fid[0] & this_fid[1])
    return fid

def isTPCE(df):
    return df.tpc <= 1


In [ ]:
# use the external input to build the t0
ccross_t0_E = data.hit_max_time_p2_tpcE - tcathode_E
ccross_t0_W = data.hit_max_time_p2_tpcW - tcathode_W

data["ccross_t0"] = ((ccross_t0_E + ccross_t0_W) / 2.) * tick_period

has_t0 = True

In [ ]:
ICARUS_FV_STOPPING = [{
        "xmin": ICARUS_AV[0]["xmin"] + 25,
        "ymin": ICARUS_AV[0]["ymin"] + 25,
        "zmin": ICARUS_AV[0]["zmin"] + 25,
        "xmax": ICARUS_AV[0]["xmax"] - 25,
        "ymax": ICARUS_AV[0]["ymax"] - 25,
        "zmax": ICARUS_AV[0]["zmax"] - 25,
    },
]

In [ ]:
stopping_track = ~InFV([data.start_x, data.start_y, data.start_z]) &\
    InFV([data.end_x, data.end_y, data.end_z], ICARUS_FV_STOPPING) &\
    (data.dir_y < 0.) &\
    has_t0 &\
    (data.hit_min_time_p2_tpcE > 200) & (data.hit_max_time_p2_tpcE < NTICKS - 200) &\
    (data.hit_min_time_p2_tpcW > 200) & (data.hit_max_time_p2_tpcW < NTICKS - 200)
    #&\
    #(plane2_stopcalo.dqdx_corr.groupby("entry").first() / plane2_stopcalo.dqdx_corr.groupby("entry").last() > 2)

stopping_track.name="stopping"
stopping_track.index.name="entry"

stopping_track.sum() / stopping_track.size

In [ ]:
plane2branches = [
    "dqdx",
    "time",
    "tpc",
    "pitch"
]
plane2branches = ["hits2.%s" % s for s in plane2branches]

In [ ]:
plane2_stopcalo = treeE.arrays(["hits2.rr"], library="pd")#, entrystop=10000)
plane2_oncalo = ~np.isnan(plane2_stopcalo["hits2.rr"]) & (plane2_stopcalo["hits2.rr"] < 200) & (plane2_stopcalo["hits2.rr"] > 0.)
plane2_stopcalo = plane2_stopcalo[plane2_oncalo]

for b in plane2branches:
    bdf = treeE.arrays([b], library="pd")
    bdf = bdf[plane2_oncalo]
    plane2_stopcalo = plane2_stopcalo.merge(bdf, left_index=True, right_index=True, validate="one_to_one")
    del bdf

In [ ]:
plane2_stopcalo.columns = [s.split(".")[1] for s in plane2_stopcalo.columns]
plane2_stopcalo = plane2_stopcalo.sort_values(by=["entry", "rr"])

In [ ]:
# Guard bad dqdx, only save tracks for which we have a ccross t0
plane2_stopcalo = plane2_stopcalo[~np.isnan(plane2_stopcalo.dqdx)]
plane2_stopcalo = plane2_stopcalo.join(data.ccross_t0)
plane2_stopcalo = plane2_stopcalo[~np.isnan(plane2_stopcalo.ccross_t0)]

t0 = 0
def exp(t, *p):
    A,tau = p
    return A*np.exp(-(t - t0)/tau)

plane2_stopcalo["tpcE"] = isTPCE(plane2_stopcalo)
plane2_stopcalo.time = (plane2_stopcalo.time * tick_period - plane2_stopcalo.ccross_t0 - tanode*tick_period) / 1000.

plane2_stopcalo["dqdx_corr"] = plane2_stopcalo.dqdx * exp(plane2_stopcalo.time, 1., -electron_lifetime_tpcE)
plane2_stopcalo.loc[~plane2_stopcalo.tpcE, "dqdx_corr"] = plane2_stopcalo.dqdx[~plane2_stopcalo.tpcE] * exp(plane2_stopcalo.time[~plane2_stopcalo.tpcE], 1., -electron_lifetime_tpcW)

plane2_stopcalo = plane2_stopcalo.drop(columns="ccross_t0")

In [ ]:
Rcut = 10 #cm
dqdxcut = 1.1e3
stopping_track_wcalo = stopping_track &\
    (plane2_stopcalo.dqdx[plane2_stopcalo.rr < 5].groupby(level=0).median() > dqdxcut)

#& (fitvar > 25) & (data.exp_fit_R < Rcut)

stopping_track_wcalo.name="stopping"
stopping_track_wcalo.index.name="entry"

stopping_track_wcalo.sum() / stopping_track_wcalo.size

In [ ]:
if "stopping" in plane2_stopcalo.columns:
    plane2_stopcalo = plane2_stopcalo.drop(columns=["stopping"])
plane2_stopcalo = plane2_stopcalo.join(stopping_track_wcalo)

plane2_stopcalo = plane2_stopcalo[plane2_stopcalo.stopping]
plane2_stopcalo = plane2_stopcalo.drop(columns=["stopping"])

In [ ]:
MODA = 0.930
MODB = 0.212
Wion = 1e3 / 4.237e7
Efield = 0.5

def recombination_corr(dQdx):
    alpha = MODA
    beta = MODB / (LAr_density_gmL * Efield)
    
    dEdx = (np.exp(beta * Wion * dQdx) - alpha) / beta
    return dEdx

In [ ]:
def landau_gaus(X, *p):
    mpv, eta, sigma, A = p
    if sigma >= 100*eta:
        sigma = eta
    return pylandau.langau(X, mpv=mpv, eta=eta, sigma=sigma, A=A)

In [ ]:
plane2_stopcalo["dedx"] = recombination_corr(plane2_stopcalo.dqdx_corr * 81.27)

In [ ]:
binx = np.linspace(0., 200., 201)
biny = np.linspace(0.5, 6, 61)

varx = plane2_stopcalo.rr
vary = plane2_stopcalo.dedx

_ = plt.hist2d(varx, vary, bins=[binx, biny])#, norm=colors.LogNorm())
plt.colorbar()
plt.xlabel("Residual Range [cm]")
plt.ylabel("dE/dx [ADC/cm]")


In [ ]:
rrs = np.linspace(5., 200., 40)
# rrs = np.linspace(150, 155., 2)
rrhis = rrs[1:]
rrlos = rrs[:-1]
RRs = (rrhis + rrlos)/2.
bins = np.linspace(0, 6, 61)
bin_centers = (bins[1:] + bins[:-1]) / 2.

MPV_dedxs = []

langau_params = []
langau_param_errs = []

fit_scale = 100.

for i,(rrlo, rrhi) in enumerate(zip(rrlos, rrhis)):
    #if i > 5: break
    when = (plane2_stopcalo.pitch < 0.4) & (plane2_stopcalo.pitch > 0.3) &\
        (plane2_stopcalo.rr > rrlo) & (plane2_stopcalo.rr < rrhi)
    N,_ = np.histogram(plane2_stopcalo.dedx[when], bins=bins)
    
    p0 = [1.5*fit_scale, 0.1*fit_scale, 0.1*fit_scale, np.max(N)]
    popt, perr = curve_fit(landau_gaus, bin_centers * fit_scale, N, p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N), 1))
    MPV_dedxs.append(popt[0] / fit_scale)
    langau_params.append(popt)
    langau_param_errs.append(perr)
    
MPV_dedxs = np.array(MPV_dedxs)

In [ ]:
langau_params_unscaled = []
for i in range(len(langau_params)):
    langau_params_unscaled.append( np.hstack([langau_params[i][:3] / fit_scale, [langau_params[i][3]]]) )

In [ ]:
# Plot fits

for IND in range(len(RRs)):
    plt.figure(IND)
    when = (plane2_stopcalo.pitch < 0.4) & (plane2_stopcalo.pitch > 0.3) &\
            (plane2_stopcalo.rr > rrlos[IND]) & (plane2_stopcalo.rr < rrhis[IND])

    plt.hist(plane2_stopcalo.dedx[when], bins=bins, label="Data")
    plt.plot(bin_centers, landau_gaus(bin_centers * fit_scale, *langau_params[IND]), label="Landau$\circledast$Gaussian Fit")
    plt.legend(fontsize=14)
    plt.text(0.5, 0.2, "%.0f < R.R. < %.0f cm\n0.3 < pitch < 0.4 cm\n\nMPV = %.2f MeV/cm\n$\eta$ = %.2f MeV/cm\n$\sigma$ = %.2f MeV/cm" % tuple([rrlos[IND], rrhis[IND]] + list(langau_params_unscaled[IND][:3])),
            fontsize=14, transform=plt.gca().transAxes)
    
    plt.xlabel("dE/dx [MeV/cm]")
    plt.ylabel("# Depositions")
    if dosave: plt.savefig(savedir + "dedx_langau_fit_RR%.0f_%.0f.png" % (rrlos[IND], rrhis[IND]))

In [ ]:
error = 0.01

In [ ]:
plt.plot(RRs, RR2DEDX(RRs), label="Prediction")
plt.errorbar(RRs, MPV_dedxs, yerr=MPV_dedxs*error, xerr=2.5, ls="none", label="Data Measured M.P.V.")
plt.legend()
plt.xlabel("Residual Range [cm]")
plt.ylabel("dE/dx [MeV/cm]")
plt.xlim([50, 200])
plt.ylim([1.5, 1.8])

if dosave: plt.savefig(savedir + "Edx_predictionVdata.png")

In [ ]:
IND = 30

when = (plane2_stopcalo.pitch < 0.4) & (plane2_stopcalo.pitch > 0.3) &\
            (plane2_stopcalo.rr > rrlos[IND]) & (plane2_stopcalo.rr < rrhis[IND])
plt.figure(0)
N,_,_ = plt.hist(plane2_stopcalo.dedx[when], bins=bins)

p0 = [1.5*fit_scale, 0.1*fit_scale, 0.1*fit_scale, np.max(N)]
popt, perr = curve_fit(landau_gaus, bin_centers * fit_scale, N, p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N), 1))
print(popt[0] / fit_scale)
plt.plot(bin_centers, landau_gaus(bin_centers * fit_scale, *popt))


dqdx_bins = np.linspace(200, 2000, 73)
dqdx_bin_centers = (dqdx_bins[1:] + dqdx_bins[:-1]) / 2.
plt.figure(1)
N,_,_ = plt.hist(plane2_stopcalo.dqdx_corr[when], bins=dqdx_bins)

p0 = [600, 35, 100, np.max(N)]
popt, perr = curve_fit(landau_gaus, dqdx_bin_centers, N, p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N), 1))
plt.plot(dqdx_bin_centers, landau_gaus(dqdx_bin_centers, *popt))

print(recombination_corr(popt[0] * 81.27))